In [58]:
# Model parameters
vocab_size = 92
n_embd = 384
block_size = 256
batch_size = 64
dropout=0.2
n_head = 4
n_layer = 4

device = 'cpu' 

In [59]:
with open('MI_Fallout.txt', 'r', encoding='utf-8') as f:
    lines = f.readlines()
    lines = [line for line in lines if not line.strip().startswith(tuple('0123456789*'))]

    
    

In [60]:
text = ''.join(lines)
print(text)


BLACK


And from out of the blackness, a familiar voice.
VOICE (O.S.)
Do you, Ethan, take Julia to be
your lawfully wedded wife?
FADE UP to reveal ETHAN HUNT, standing in the impossibly
bright sun, wearing a suit, smiling warmly.
I do.

ETHAN

PULL BACK TO REVEAL: A BRIDE in a simple white dress and a
wedding veil. We can just make out her lovely face. A face we
might recognize. She smiles. Nervous, happy, deeply in love.
VOICE (O.S.)
To have and to hold. To love,
cherish, honor and protect.
I do.

ETHAN

VOICE (O.S.)
To shield from terrors known and
unknown. To lie, to deceiveWhat?

ETHAN

VOICE
To live a double life. To
needlessly place her in harms way,
fail to prevent her abductionWh- No.

ETHAN

VOICE (O.S.)
To rob her of her innocence, erase
her identity, force her into
hiding, take away all she's known
in a selfish, futile, fleeting
attempt to escape your true self.
Stop.

ETHAN

VOICE (O.S.)
And Julia... Do you choose to
accept?

 

Don’t.
I do.
Don’t.


ETHAN
JULIA
ETHAN

RE

In [61]:
vocab = sorted(list(set(text)))
print(vocab)
vocab_size = len(vocab)
print('Vocabulary size:', len(vocab))

['\n', '\x0c', ' ', '!', '#', '&', "'", '(', ')', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '©', 'Ç', 'É', 'ç', 'é', 'ê', 'ö', '‘', '’', '‚', '“', '”']
Vocabulary size: 92


In [62]:
stoi = {ch: i for i, ch in enumerate(vocab)}
itos = {i: ch for i, ch in enumerate(vocab)}
encode = lambda s : [stoi[ch] for ch in s]
decode = lambda l : ''.join([itos[i] for i in l])
l = encode('Chicken')
print(decode(l))

# use tiktoken

Chicken


In [63]:
import torch 
#import tiktoken




In [64]:
#enc = tiktoken.get_encoding('gpt2')
#print(enc.encode('Chicken'))

In [65]:
#print(enc.encode('something else'))

In [66]:
data = torch.tensor(encode(text), dtype = torch.long)
print(data.shape, data.dtype)
print(data[:100])

# not using tiktoken here

torch.Size([178728]) torch.int64
tensor([ 0, 27, 37, 26, 28, 36,  0,  0,  0, 26, 67, 57,  2, 59, 71, 68, 66,  2,
        68, 74, 73,  2, 68, 59,  2, 73, 61, 58,  2, 55, 65, 54, 56, 64, 67, 58,
        72, 72,  9,  2, 54,  2, 59, 54, 66, 62, 65, 62, 54, 71,  2, 75, 68, 62,
        56, 58, 11,  0, 47, 40, 34, 28, 30,  2,  7, 40, 11, 44, 11,  8,  0, 29,
        68,  2, 78, 68, 74,  9,  2, 30, 73, 61, 54, 67,  9,  2, 73, 54, 64, 58,
         2, 35, 74, 65, 62, 54,  2, 73, 68,  2])


In [67]:
n = int(0.8*len(data))
train = data[:n]
test = data[n:]


In [68]:


def get_batch(split):
    data = train if split == "train" else test
    index = torch.randint(len(data)-block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in index])
    y = torch.stack([data[i+1:i+block_size+1] for i in index])
    return x,y

xb,yb = get_batch("train")
print(xb.shape)
print(xb)
print(yb)

torch.Size([64, 256])
tensor([[ 0,  0,  0,  ..., 61, 68, 74],
        [73, 72,  2,  ..., 58, 88, 72],
        [72,  2, 62,  ..., 66, 62, 56],
        ...,
        [33, 40, 46,  ..., 54,  2, 57],
        [74, 72, 73,  ..., 74, 73, 61],
        [68, 67, 67,  ..., 66, 58,  2]])
tensor([[ 0,  0, 34,  ..., 68, 74, 60],
        [72,  2, 73,  ..., 88, 72,  2],
        [ 2, 62, 67,  ..., 62, 56, 71],
        ...,
        [40, 46, 44,  ...,  2, 57, 68],
        [72, 73, 68,  ..., 73, 61, 58],
        [67, 67, 54,  ..., 58,  2, 73]])


In [69]:
import torch
import torch.nn as nn
from torch.nn import functional as F

class Head(nn.Module):

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd,head_size,bias=False)
        self.query = nn.Linear(n_embd,head_size,bias=False)
        self.value = nn.Linear(n_embd,head_size,bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout= nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)
        q = self.query(x)

        wei = q @ k.transpose(-2,-1) * (C ** -0.5)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim = -1)
        wei = self.dropout(wei)
        v = self.value(x)
        out  = wei @ v

        return out

class FeedForward(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4*n_embd),
            nn.ReLU(),
            nn.Linear(4*n_embd, n_embd),
            nn.Dropout(dropout)
        )
    
    def forward(self, x):
        return self.net(x)

class MultiHead(nn.Module):
    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout= nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class Block(nn.Module):
    
    def __init__(self, n_embd, n_head):
        super().__init__()
        head_size = n_embd//n_head
        self.sa = MultiHead(n_head, head_size)
        self.ffw = FeedForward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self,x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffw(self.ln2(x))
        return x


# class LayerNorm:
#     def __init__(self, dim, eps = 1e-5, momentum = 0.1):
#         self.eps = eps
#         self.momentum = momentum
#         self.gamma = torch.ones(dim)
#         self.beta = torch.zeros(dim)

#     def __call__(self,x):
#         xmean = x.mean(1, keepdim=True)
#         xvar = x.var(1, keepdim=True)
#         xhat = (xmean-xvar)/torch.sqrt(xvar + self.eps)
#         self.out = self.gamma*xhat + self.beta
#         return self.out
    
#     def parameters(self):
#         return [self.gamma, self.beta]


class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size,n_embd)
        self.position_embeddin_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(
            Block(n_embd, n_head=4),
            Block(n_embd, n_head=4),
            Block(n_embd, n_head=4),
            nn.LayerNorm(n_embd)
        )
        # self.sa_heads = MultiHead(4,n_embd//4)
        # self.ffw = FeedForward(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B,T = idx.shape
        tokenembd = self.token_embedding_table(idx)
        posemb = self.position_embeddin_table(torch.arange(T, device = idx.device))
        x = tokenembd + posemb
        x = self.blocks(x)
        logits = self.lm_head(x)

        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits,loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            idx_cond = idx[:, -block_size:]
            logits,loss = self(idx_cond)
            logits = logits[:,-1,:]
            probs = F.softmax(logits, dim = -1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx,idx_next), dim = 1)
        return idx




    
m = BigramLanguageModel().to(device)
out, loss = m(xb,yb)
print(out.shape)
print(loss)

idx = torch.zeros((1,1), dtype=torch.long).to(device)
print(decode(m.generate(idx, max_new_tokens=100)[0].tolist()))

torch.Size([16384, 92])
tensor(4.6880, grad_fn=<NllLossBackward0>)

vmr ‚v;,&LPPN:J/L! YH4t
[IxyME”#f 9 rYR:/0&é#SviÇbVk7j0oq/E0‚q#l89‚ou1 PPoc?44BypF(bMqe nHUb?W És‘ÉA


In [76]:


optimizer = torch.optim.AdamW(m.parameters(), lr = 1e-3)

for steps in range(100):
    xb,yb = get_batch("train")

    logits,loss = m(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

1.313081979751587


In [ ]:
idx = torch.zeros((1, 1), dtype=torch.long).to(device)

output_idx = m.generate(idx, max_new_tokens=1000)

print(decode(output_idx[0].tolist()))




LANE
Areales Luther we meday. That’s is as Benji and moore anmergerNG.
SLATHER
Luther He doing DIVE Lark and unnects, Ilsa
bombusid little. And you’re
minify need looking. Benji takes to
the dator.


 



ETHAN
Ethan’s eyes down to be forge that
quippeace the nummen to .



I down see are inction under of Ethan a knicked doorning back beginmen and
outsiders of Lark’s micondone gon bed un.
Ethan his has this take tryied. Theam old was desplay infor no hands indered bands anvoiced
ticening off the 20Me. Acchway, alking down smill their in from an heli,
nhicars and running HAO, but hourtiable is
beeining.
BENJI
And Meccuffces. On tise.
Ethan launnes steps the phone on the into he tinte
heartles, bridge who that’s one don’t ol
that...
Special courthate. ETHAN
(what’s realing this when front. Prasside is the Thugle of Ethan Ridies. Bullet, to only runs the
enee. He runneles glives oper.
Benji. Benji abarroticountil let his rogess. In
mosting. Lark sporristraugy. This have in Lane pursens